# Load a genome to the HealthOmics Sequence Store
---
This notebook demonstrates how to download a public genome to a HealthOmics sequence store. This is 
in preparation for later pretraining of a genomic language model (HyenaDNA).

## Prerequisites

1. In order to download and process the data you should make sure that this notebook has access to at least
64Gb of disc storage.
2. The notebook needs to have permissions to access S3. To add these permissions, follow [these instructions](https://docs.aws.amazon.com/omics/latest/dev/manage-reference-store.html). You
can see what your notebook's execution role by running this: `print(sagemaker.get_execution_role())`

In [ ]:
%pip install -qU transformers

In [ ]:
from pathlib import Path
import importlib
from functools import partial
from time import sleep
import json
import sys
import os

cwd = os.getcwd()
if cwd.endswith("/hyena-DNA"):
    repo_base = cwd.rpartition("/")[0]
elif cwd.endswith("healthomics-seq-store"):
    repo_base = cwd
else:
    raise Exception(f"port me: {cwd}")
print(f"repo base: {repo_base}")
!cp {repo_base}/utilities.py {repo_base}/evo-model/scripts
sys.path.append(repo_base)
import utilities as u
# to reload the utilities without restarting the kernel, use this: importlib.reload(u)

import boto3
import sagemaker

In [ ]:
sequence_store_name = "mouse genome"
# This is where we upload the compressed FASTQ files:
bucket_name = "sgh-misc"
prefix = "data/mouse/"

In [ ]:
import_job_role_arn = "arn:aws:iam::111918798052:role/OmicsImportRole"

In [ ]:
omics = boto3.client("omics")
s3 = boto3.client("s3")

First, we download a [mouse reference genome](https://www.ncbi.nlm.nih.gov/datasets/genome/GCA_921998355.2/)
from Genbank onto the local disk (this should take less than a minute with a broadband connection). There should
be one (compressed) FASTA file per chromosome.

In [ ]:
!wget -P ~/mouse/ -r -nH --cut-dirs=11 --no-parent ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/vertebrate_mammalian/Mus_musculus/latest_assembly_versions/GCA_921998355.2_A_J_v3/GCA_921998355.2_A_J_v3_assembly_structure/Primary_Assembly/assembled_chromosomes/FASTA/

If the above command finishes successfully then you should be able to verify the results
and see something similar to this:
```bash
sh-4.2$ ls -l ~/mouse
total 703768
-rw-rw-r-- 1 ec2-user ec2-user 36522179 Jul 19  2022 chr10.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 34213890 Jul 19  2022 chr11.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 32615588 Jul 19  2022 chr12.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 32962659 Jul 19  2022 chr13.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 32646782 Jul 19  2022 chr14.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 29093156 Jul 19  2022 chr15.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 27225255 Jul 19  2022 chr16.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 26229898 Jul 19  2022 chr17.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 25313610 Jul 19  2022 chr18.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 16757414 Jul 19  2022 chr19.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 55286326 Jul 19  2022 chr1.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 50752623 Jul 19  2022 chr2.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 44959307 Jul 19  2022 chr3.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 42502452 Jul 19  2022 chr4.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 41498510 Jul 19  2022 chr5.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 41907927 Jul 19  2022 chr6.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 38157680 Jul 19  2022 chr7.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 35790409 Jul 19  2022 chr8.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 34764818 Jul 19  2022 chr9.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 41422059 Jul 19  2022 chrX.fna.gz
```

In [ ]:
data = Path.home() / "SageMaker" / "mouse"
if not data.exists():
    data = Path.home() / "mouse"
data

Next, we uncompress the compressed FASTA (".fna.gz") files to create FASTA (".fna") files:

In [ ]:
fasta_files = u.convert_directory(data, suffix=".fna.gz",
                                  convertor=partial(u.gunzip_file,
                                                    suffix=".gz"))

Now, we convert those FASTA files into FASTQ files:

In [ ]:
fastq_files = u.convert_directory(data, suffix=".fna",
                                  convertor=u.convert_fasta_file_to_fastq)

And then we compress these FASTA files:

In [ ]:
compressed_fq_files = u.convert_directory(data, suffix=".fq",
                                          convertor=u.gzip_file)

Next, we upload the files to S3

In [ ]:
s3_uris = []
for file in compressed_fq_files:
    key = f"{prefix}{file.name}"
    s3.upload_file(file, bucket_name, key)
    s3_uri = f"s3://{bucket_name}/{key}"
    print(s3_uri)
    s3_uris.append(s3_uri)
print("Done")

Next, we create a sequence store in HealthOmics

In [ ]:
seq_store_resp = omics.create_sequence_store(
    name=sequence_store_name,
    description="GCA_921998355.2_A_J_v3"
)
seq_store_id = seq_store_resp["id"]
print(f"Sequence store ID: {seq_store_id}")

Next, we load our FASTQ files into this new sequence store

In [ ]:
import_job_resp = omics.start_read_set_import_job(
    sequenceStoreId=seq_store_id,
    roleArn=import_job_role_arn,
    sources=[
        {
            "sourceFiles": {"source1": s3_uri},
            "sourceFileType": "FASTQ",
            "subjectId": "N/A",
            "sampleId": "N/A",
        }
        for s3_uri in s3_uris
    ]
)
import_job_id = import_job_resp["id"]
print(f"Import job ID: {import_job_id}")

We now wait for these read sets to be imported into the sequence store. This typically takes about an hour.

In [ ]:
%%time
while True:
    job_list_response = omics.list_read_set_import_jobs(maxResults=100,
                                                        sequenceStoreId=seq_store_id)
    import_jobs = [job for job in job_list_response["importJobs"] if job["id"] == import_job_id]
    [status] = [job["status"] for job in import_jobs] # filtered on job id, so should only be one
    print(f"Status of import job {import_job_id} is {status}")
    if status not in {"SUBMITTED", "IN_PROGRESS"}:
        break
    sleep(5*60)
print("Done")

## Integration with the training notebook

First, check your AWS CLI version, it must be >= 2.15.20:

In [ ]:
!/usr/local/bin/aws --version

If your version of `aws-cli` is 1.X then go [here](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html) to see how to upgrade to 2.X.

In [ ]:
!/usr/local/bin/aws omics get-sequence-store --id {seq_store_id} > /tmp/seq-store.json

In [ ]:
seq_store_info = json.loads(Path("/tmp/seq-store.json").read_text())
s3AccessPoint = seq_store_info["s3Access"]["s3Uri"]

In [ ]:
print(f"Sequence store ID: {seq_store_id}")
print(f"This sequence store's s3 access point: {s3AccessPoint}")